# OUTLINE: 
- Why do we need to use typing? 
- What is the basic flow to work with user input 
- Primitive types 
- Non-primitive types
- Custom types 
- Alias, TypeVar, Generic (Intermediate lvl part)
- How to use types and `pydantic` to parse user input?

## Basic idea of working with an user input

# [user input] -> [validation] -> [validated data]

# Lets start from the primitive types 

In [1]:
def divide(a: int, b: int) -> float:
    return a/b

result = divide(10, 4)
print(result)

2.5


In [2]:
def make_header(text: str)->str:
    return f"{text.capitalize():-^20}"

result: str = make_header("header1")
print(result)

------Header1-------


In [3]:
def boolean_and(a: bool, b: bool)->bool:
    return a and b

result: bool = boolean_and(True, False)
print(result)

False


# Ok. Now lets consider embedded Non-primitive data types

In [4]:
def custom_join(data: list[int]) -> str:
    str_list = [str(item) for item in data]
    return " ".join(str_list)

result =  custom_join([1,2,3,4,5]) 
print(result)

1 2 3 4 5


In [5]:

def custom_join(data: set[int]) -> str:
    str_list = [str(item) for item in data]
    return " ".join(str_list)

result: str =  custom_join({1,2,3,4,5}) 
print(result)

1 2 3 4 5


In [6]:
def custom_join(data: tuple[int]) -> str:
    str_list = [str(item) for item in data]
    return " ".join(str_list)

result: str =  custom_join((1,2,3,4,5)) 
print(result)

1 2 3 4 5


In [7]:
def custom_join(data: dict[str, int]) -> str:
    str_list = [f"{key}&{value}" for key, value in data.items()]
    return " ".join(str_list)

result: str =  custom_join({"foo": 1, "boo": 2, "zoo": 3}) 
print(result)

foo&1 boo&2 zoo&3


# Type is also an object

In [8]:
from typing import Union, Optional

PRIMITIVE_TYPE = Union[bool, str, int, float]

def custom_join(data: Optional[list[PRIMITIVE_TYPE]]) -> Optional[str]:
    if data is None:
        return None
    
    str_list = [str(item) for item in data]
    return " ".join(str_list)

result: str =  custom_join([True,"test",3,4.5]) 
print(result)

True test 3 4.5


# Soo. How to create our custom type? 

In [9]:
from typing import Protocol

class IRectangle(Protocol):
    length: float
    width: float
    area: float
    perimeter: float
    
   

class Rectangle(IRectangle): 
    def __init__(self, length: float, width: float) -> None:
        self.length = length
        self.width = width
    
    @property
    def area(self)->float:
        return self.length*self.width
    
    @property
    def perimeter(self)->float:
        return self.length*2 + self.width*2
    
    def __str__(self) -> str:
        return f"length={self.length}\nwidth={self.width}\narea={self.area}\nperimeter={self.perimeter}"
    

def custom_sum(rectangles: list[IRectangle]) -> IRectangle:
    length = sum([rectangle.length for rectangle in rectangles])
    width = sum([rectangle.width for rectangle in rectangles])
    
    return Rectangle(length, width)

rectangles = [Rectangle(10, 2), Rectangle(5, 2), Rectangle(10, 3)]

rectangle = custom_sum(rectangles)

print(rectangle)

length=25
width=7
area=175
perimeter=64


# if type is an object can we use it as a variable? 
# - YES

# Generics: 

In [11]:
from typing import TypeVar, Protocol
from dataclasses import dataclass
from math import pi

class GeometryObject(Protocol):
    area: float
    perimeter: float
    
@dataclass
class Rectangle:
    length: float
    width: float
    
    @property
    def area(self)->float:
        return self.length*self.width
    
    @property
    def perimeter(self)->float:
        return self.length*2 + self.width*2

@dataclass()
class Circle:
    radius: float
    
    @property
    def area(self)->float:
        return self.radius**2 * pi
    
    @property
    def perimeter(self)->float:
        return self.radius*pi*2 

GEOMETRY_OBJ_TYPE = TypeVar('GEOMETRY_OBJ_TYPE', bound=GeometryObject)

def get_biggest_object(geometry_objs: list[GEOMETRY_OBJ_TYPE]) -> GEOMETRY_OBJ_TYPE:
    return max(geometry_objs, key=lambda x: x.area)   

circles = [Circle(1),Circle(3),Circle(2),Circle(2.5)]
rectangles = [Rectangle(10, 2), Rectangle(5, 2), Rectangle(10, 3)]

biggest_rectangle = get_biggest_object(rectangles)
biggest_circle = get_biggest_object(circles)

print(f"biggest_rectangle: {biggest_rectangle}")
print(f"biggest_circle: {biggest_circle}")

biggest_rectangle: Rectangle(length=10, width=3)
biggest_circle: Circle(radius=3)


In [12]:
from typing import Generic, TypeVar, Generic, Tuple, Protocol # noqa
from dataclasses import dataclass
from math import pi # noqa

class GeometryObject(Protocol):
    area: float
    perimeter: float
    
@dataclass
class Rectangle:
    length: float
    width: float
    
    @property
    def area(self)->float:
        return self.length*self.width
    
    @property
    def perimeter(self)->float:
        return self.length*2 + self.width*2

@dataclass()
class Circle:
    radius: float
    
    @property
    def area(self)->float:
        return self.radius**2 * pi
    
    @property
    def perimeter(self)->float:
        return self.radius*pi*2 

GEOMETRY_OBJ_TYPE = TypeVar('GEOMETRY_OBJ_TYPE', bound=GeometryObject)

class ObjectContainer(Generic[GEOMETRY_OBJ_TYPE]):
    def __init__(self) -> None:
        self._container: dict[int, GEOMETRY_OBJ_TYPE] = {}
    
    def set_item(self, geometry_obj: GEOMETRY_OBJ_TYPE) -> int:
        uid = hash(str(len(self._container)))
        self._container[uid] = geometry_obj
        return uid
    
    def get_item(self, uid: int)-> GEOMETRY_OBJ_TYPE:
        return self._container[uid]
    
    def get_total_area(self)->float:
        return sum([item.area for item in self._container.values()])
    
    def get_total_perimeter(self)->float:
        return sum([item.perimeter for item in self._container.values()])

def create_container(items: list[GEOMETRY_OBJ_TYPE])-> Tuple[ObjectContainer[GEOMETRY_OBJ_TYPE], list[int]]:
    container: ObjectContainer[GEOMETRY_OBJ_TYPE] = ObjectContainer()
    uids = []
    for item in  items:
        uid = container.set_item(item)
        uids.append(uid)

    return container, uids

circle_container, circle_uids = create_container([Circle(1),Circle(3),Circle(2),Circle(2.5)])    
rectangle_container, rectangle_uids = create_container([Rectangle(10, 2), Rectangle(5, 2), Rectangle(10, 3)])

circle = circle_container.get_item(circle_uids[0])
rectangle = rectangle_container.get_item(rectangle_uids[0])

print(f"Rectangle: {rectangle}")
print(f"{rectangle_container.get_total_area()=}")

print(f"Circle: {circle}")
print(f"{circle_container.get_total_area()=}")



Rectangle: Rectangle(length=10, width=2)
rectangle_container.get_total_area()=60
Circle: Circle(radius=1)
circle_container.get_total_area()=63.61725123519331


# AND FINALLY THE REASON WHY WE ARE HERE!!! 
# HOW TO USE ALL OF THIS S**** TO WRITE A PARSER?

In [13]:
from typing import Any, TypeVar, Generic
from xml.dom.minidom import Node

from pydantic import BaseModel

NodePropertyType = TypeVar("NodePropertyType")
LinkPropertyType = TypeVar("LinkPropertyType")

class BasicPropertiesType(BaseModel):
    name: str


class PackagePropertiesType(BaseModel):
    name: str
    package: str
    launch_file: str
    parameters: dict[str, Any] | None = None


class StorageCellPropertiesType(BaseModel):
    name: str
    z: int
    x: int
    y: int


class TF2RelationshipType(BaseModel):
    x_offset: float
    y_offset: float
    z_offset: float


class StorageCellRelationshipType(BaseModel):
    row: int
    column: int


class LinkType(BaseModel, Generic[LinkPropertyType]):
    parent: str
    properties: LinkPropertyType | None = None


class LinksType(BaseModel):
    managed_by: LinkType[None] | None = None
    controlled_by:LinkType[None] | None = None
    attached_to: LinkType[TF2RelationshipType] | None = None
    supervised_by: LinkType[None] | None = None
    belong_to: LinkType[StorageCellRelationshipType] | None = None

class NodeType(BaseModel, Generic[NodePropertyType]):
    links: LinksType | None = None
    properties: NodePropertyType

class UserConfig(BaseModel):
    version: str

    SUPERVISOR: NodeType[BasicPropertiesType]
    TF2: NodeType[BasicPropertiesType]
    HARDWARE_BRIDGE: NodeType[PackagePropertiesType]

    MANAGER: list[NodeType[PackagePropertiesType]]
    CARRIER: list[NodeType[PackagePropertiesType]]
    TOOL: list[NodeType[PackagePropertiesType]]
    CONTROLLER: list[NodeType[PackagePropertiesType]]

    STORAGE: list[NodeType[BasicPropertiesType]]
    STORAGE_CELL: list[NodeType[StorageCellPropertiesType]]


In [14]:
import yaml
from pathlib import Path

path_to_yaml = Path(__name__).resolve().parent.joinpath("config_file.yaml")

yaml_conf = {}
with open(path_to_yaml) as f:
    yaml_conf = yaml.safe_load(f)

config = UserConfig(**yaml_conf)

In [16]:
config.MANAGER[0].properties


PackagePropertiesType(name='Manger', package='move_manager_code', launch_file='move_manager_launch.py', parameters=None)